In [1]:
import setup
setup.init_django()

0pEnqPfhRk7bYu5LSa7ppMuKfqr21kck


In [2]:
import helpers.clients as helper_clients

In [3]:
company_name = "NVDA"
company_ticker = "NVDA"
multiplier = 1
from_date = "2025-01-01"
to_date = "2025-01-20"

In [4]:
client = helper_clients.PolygonAPIClient(
    ticker=company_ticker,
    multiplier=multiplier,
    from_date=from_date,
    to_date=to_date
)
dataset = client.get_stock_data()

In [5]:
len(dataset)

10545

In [6]:
new_stock_data = dataset[0]
new_stock_data

{'open_price': 132.95,
 'close_price': 134.78,
 'high_price': 135.29,
 'low_price': 132.95,
 'number_of_trades': 1493,
 'volume': 228375,
 'volume_weighted_average': 133.7491,
 'raw_timestamp': 1735808400000,
 'time': datetime.datetime(2025, 1, 2, 9, 0, tzinfo=<UTC>)}

In [7]:
from market.models import Company, StockQuote

In [8]:
company_obj, created = Company.objects.get_or_create(name=company_name, ticker=company_ticker)
# company_obj, created = Company.objects.create(name=company_name, ticker=company_ticker)
# company_obj, created = Company.objects.get(name=company_name, ticker=company_ticker)


In [9]:
company_obj

<Company: Company object (12)>

In [10]:
new_quotes = []
for data in dataset:
    new_quotes.append(
        StockQuote(company = company_obj, **data)
    )
StockQuote.objects.bulk_create(new_quotes, ignore_conflicts=True)

[<StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: St

In [11]:
# new_quotes = []

# for data in dataset:
#     new_quotes.append(
#         StockQuote(company=company_obj, **data)
#     )

In [12]:
StockQuote.objects.all()

<QuerySet [<StockQuote: StockQuote object (137223)>, <StockQuote: StockQuote object (137224)>, <StockQuote: StockQuote object (137225)>, <StockQuote: StockQuote object (137226)>, <StockQuote: StockQuote object (137227)>, <StockQuote: StockQuote object (137228)>, <StockQuote: StockQuote object (137229)>, <StockQuote: StockQuote object (137230)>, <StockQuote: StockQuote object (137231)>, <StockQuote: StockQuote object (137232)>, <StockQuote: StockQuote object (137233)>, <StockQuote: StockQuote object (137234)>, <StockQuote: StockQuote object (137235)>, <StockQuote: StockQuote object (137236)>, <StockQuote: StockQuote object (137237)>, <StockQuote: StockQuote object (137238)>, <StockQuote: StockQuote object (137239)>, <StockQuote: StockQuote object (137240)>, <StockQuote: StockQuote object (137241)>, <StockQuote: StockQuote object (137242)>, '...(remaining elements truncated)...']>

In [13]:
batch_size = 1000
for i in range(0, len(dataset), batch_size):
    batch_chunk = dataset[i:i+batch_size]
    chunked_quotes = []
    for data in batch_chunk:
        chunked_quotes.append(
            StockQuote(company=company_obj, **data)
        )
    StockQuote.objects.bulk_create(chunked_quotes, ignore_conflicts=True)

In [14]:
StockQuote.objects.all().count()

59443